In [1]:
# !wget http://files.grouplens.org/datasets/movielens/ml-latest.zip

In [2]:
# !unzip -o ml-latest.zip

In [4]:
# %cd ml-latest

/content/ml-latest


In [1]:
# !pip install lightfm

In [5]:
# !mkdir exp

In [4]:
from scipy import sparse
from lightfm import LightFM
from tqdm import tqdm
import sys

## Split data by time

In [6]:
# train/test edge collect the timestamp < time_cut 
#   and rate > 3
train_edges, test_edges = [], []


# train_all/test_all edge collect timestamp < time_cut
#   and all rating

train_all_edges, test_all_edges = [], []
#time_cut = 1514736000
time_cut = 1527811200
with open('./ratings.csv', 'r') as f:
    next(f)
    for line in tqdm(f):
        uid, iid, rate, time = line.rstrip('\n').split(',')
        if int(time) < time_cut:
            train_all_edges.append(f"u{uid}\t{iid}\t{rate}")
            if float(rate) > 3.:
                train_edges.append(f"u{uid}\t{iid}\t{rate}")
        else:
            test_all_edges.append(f"u{uid}\t{iid}\t{rate}")
            if float(rate) > 3.:
                test_edges.append(f"u{uid}\t{iid}\t{rate}")

with open('exp/ml.train', 'w') as f:
    f.write('\n'.join(train_edges))
    f.write('\n')
with open('exp/ml.test', 'w') as f:
    f.write('\n'.join(test_edges))
    f.write('\n')
with open('exp/ml2.train', 'w') as f:
    f.write('\n'.join(train_all_edges))
    f.write('\n')
with open('exp/ml2.test', 'w') as f:
    f.write('\n'.join(test_all_edges))
    f.write('\n')


27753444it [01:16, 361710.07it/s]


In [7]:
!head exp/ml.train exp/ml.test exp/ml2.train exp/ml2.test

==> exp/ml.train <==
u1	307	3.5
u1	481	3.5
u1	1257	4.5
u1	1449	4.5
u1	2134	4.5
u1	2478	4.0
u1	3020	4.0
u1	3424	4.5
u1	3698	3.5
u1	3893	3.5

==> exp/ml.test <==
u36	25	4.0
u36	36	4.0
u36	47	4.5
u36	50	4.0
u36	111	5.0
u36	293	4.5
u36	296	4.0
u36	318	4.0
u36	337	4.0
u36	356	4.0

==> exp/ml2.train <==
u1	307	3.5
u1	481	3.5
u1	1091	1.5
u1	1257	4.5
u1	1449	4.5
u1	1590	2.5
u1	1591	1.5
u1	2134	4.5
u1	2478	4.0
u1	2840	3.0

==> exp/ml2.test <==
u36	25	4.0
u36	32	3.0
u36	36	4.0
u36	47	4.5
u36	50	4.0
u36	111	5.0
u36	223	3.0
u36	293	4.5
u36	296	4.0
u36	318	4.0


In [8]:
users = []
items = []
clicks = []

# user_map record the index -> label
# user_index record the label -> index  
user_map, user_index = {}, {}
item_map, item_index = {}, {}

sys.stderr.write('load train data ...\n')
with open("exp/ml.train") as f:
    for line in f:
        user, item, click = line.rstrip('\n').split()
        if user not in user_index:
            user_index[user] = len(user_index)
            user_map[user_index[user]] = user
        if item not in item_index:
            item_index[item] = len(item_index)
            item_map[item_index[item]] = item
      
        users.append(user_index[user])
        items.append(item_index[item])
        clicks.append(float(click))

train = sparse.coo_matrix((clicks, (users, items)))

sys.stderr.write('done !!! \n')

load train data ...
done !!! 


In [9]:
print(train)

  (0, 0)	3.5
  (0, 1)	3.5
  (0, 2)	4.5
  (0, 3)	4.5
  (0, 4)	4.5
  (0, 5)	4.0
  (0, 6)	4.0
  (0, 7)	4.5
  (0, 8)	3.5
  (0, 9)	3.5
  (1, 10)	3.5
  (1, 11)	3.5
  (1, 12)	3.5
  (1, 13)	4.5
  (1, 14)	4.0
  (1, 15)	3.5
  (1, 16)	4.5
  (1, 17)	4.0
  (1, 18)	3.5
  (1, 19)	4.0
  (1, 20)	3.5
  (1, 21)	4.0
  (2, 22)	4.0
  (2, 23)	4.0
  (2, 24)	4.0
  :	:
  (276010, 1507)	4.0
  (276010, 297)	3.5
  (276010, 298)	5.0
  (276010, 1062)	4.5
  (276010, 356)	5.0
  (276010, 364)	4.0
  (276010, 2013)	4.5
  (276010, 384)	3.5
  (276010, 2216)	4.0
  (276010, 396)	4.5
  (276010, 409)	5.0
  (276010, 424)	4.0
  (276010, 7701)	5.0
  (276010, 5578)	4.5
  (276010, 5579)	4.0
  (276010, 6639)	4.5
  (276010, 7389)	4.5
  (276010, 7407)	5.0
  (276010, 12129)	5.0
  (276010, 2037)	5.0
  (276010, 7410)	4.5
  (276010, 5605)	4.5
  (276010, 1015)	4.5
  (276010, 3394)	4.5
  (276010, 508)	4.5


In [13]:
model = LightFM(learning_rate=0.025, no_components=128, loss='warp', item_alpha=0.001, user_alpha=0.001)
sys.stderr.write('start training ...\n')
model.fit(train, epochs=100, num_threads=16, verbose=True)

start training ...

Epoch: 100%|██████████| 100/100 [47:41<00:00, 28.62s/it]


In [14]:
sys.stderr.write('save representations ...\n')
user_reps = model.get_user_representations()
item_reps = model.get_item_representations()d
rep_res = []
#rep_res.append('%d %d' % (len(user_reps[1])+len(item_reps[1]), args.dim))
for e, user_rep in enumerate(user_reps[1]):
    rep_res.append('%s\t%f 1.0 %s' % (user_map[e], user_reps[0][e], ' '.join(list(map(str, user_rep)))))
for e, item_rep in enumerate(item_reps[1]):
    rep_res.append('%s\t1.0 %f %s' % (item_map[e], item_reps[0][e], ' '.join(list(map(str, item_rep)))))

with open("exp/ml.lfm", 'w') as f:
    f.write('%s\n' % ('\n'.join(rep_res)))

save representations ...


In [27]:
!head exp/ml.lfm -n 1

u1	-1.005874 1.0 -2.5481415 -4.053117 0.9342967 -0.23839854 -2.3836365 2.500808 -1.2779369 0.72772706 0.881725 -0.11601348 -1.0355482 0.72646445 -0.04656492 1.3394637 1.8984878 -1.5686283 -1.7636406 -2.800289 1.4400064 0.6177509 1.5478159 3.5321524 -0.7459478 1.5155205 -0.20599326 1.4443252 -2.4191566 1.6636883 -0.28033775 0.665832 1.9568146 -0.10864849 -0.14182271 -0.9869911 -1.1713748 1.174228 -0.23050216 0.24290244 3.704556 1.6157317 -7.005079 -1.850436 -0.1109539 -4.0255623 -2.0795867 -0.3099979 -2.6072927 -7.3014097 2.6778626 -0.62580454 2.4130542 -0.53084314 0.071333855 -0.77788556 -1.3622967 1.8971088 -6.852613 0.54805773 0.3152457 0.890362 1.2032653 0.40478864 -0.02777551 0.9853546 -2.8206947 1.5503545 -1.2679352 -0.16621418 -1.2484174 -1.765311 0.7622128 -0.63699806 0.6783687 1.1941508 -0.65689546 0.400857 0.06446028 -1.1058619 0.103712 0.37212646 0.70208323 -2.1980374 0.20495467 0.40018085 -0.017207354 4.2750816 -0.26779494 1.7234085 1.6579169 0.23225619 1.4389279 1.1251903 -

## evaluation

In [16]:
import sys
import gzip, json
import argparse
import random
import math
import concurrent.futures
from collections import defaultdict
from math import log, sqrt
from tqdm import tqdm

In [20]:
seed = 2021
cold_item = 0
cold_user = 0
def eu_distance(v, v2):
    try:
        return -(sum( (a-b)**2 for a,b in zip(v, v2) )) # omit sqrt
    except:
        return -100.

def cosine_distance(v, v2):
    try:
        score = sum( (a*b) for a,b in zip(v, v2) ) / ( sqrt(sum(a*a for a in v)*sum(b*b for b in v2)) )
        if math.isnan(score):
            return 0
        return score
    except:
        return 0

def dot_sim(v, v2):
    try:
        score = sum( (a*b) for a,b in zip(v, v2) )
        if math.isnan(score):
            return 0
        return score
    except:
        return 0

sim = {'dot': dot_sim, 'cosine': cosine_distance}["dot"]

def process_u_query(uid):

    global sim
    global iids, cold_iids
    global train_ui, test_ui
    global train_ui, test_ui
    global embed
    global args
    item_pool = list(iids.keys())
    if cold_item:
        item_pool += list(cold_iids.keys())

    ui_results = [uid, str(len(test_ui[uid]))]

    # scoring
    ui_scores = defaultdict(lambda: 0.)
    for rid in item_pool:
        if rid in train_ui[uid]:
            continue
        if uid in embed and rid in embed:
            ui_scores[rid] += sim(embed[uid], embed[rid])
        else:
            ui_scores[rid] += 0.

    # ranking
    for rid in sorted(ui_scores, key=ui_scores.get, reverse=True):
        if rid in test_ui[uid]:
            ui_results.append('1')
        else:
            ui_results.append('0')

    return ' '.join(ui_results)

In [ ]:
print ('load train data from',"./exp/ml.train")
uids, iids = {}, {}
train_ui = defaultdict(dict)
train_counter = 0.
with open("exp/ml.train") as f:
    for line in f:
        uid, iid, target = line.rstrip().split('\t')
        train_counter += 1.
        uids[uid] = 1
        iids[iid] = 1
        train_ui[uid][iid] = 1

print ('load answer data from', "ml.test")
test_ui = defaultdict(dict)
cold_uids, cold_iids = {}, {}
test_counter = 0.
with open("./exp/ml.test") as f:
    for line in f:
        uid, iid, target = line.rstrip().split('\t')
        if uid not in uids:
            cold_uids[uid] = 1
        if iid not in iids:
            cold_iids[iid] = 1
        if iid in cold_iids and not cold_item:
            continue
        test_counter += 1.
        test_ui[uid][iid] = float(target)

print ("load embeddings from","ml.lfm")
embed = {}
with open("./exp/ml.lfm", 'r') as f:
    lines = f.readlines()
    for line in lines[:]:
        line = line.rstrip().split('\t')
        ID = line[0]
        embed[ID] = list(map(float, line[1].split(' ')))

u_queries = [u for u in test_ui if u not in cold_uids]
print('warm user:', len(u_queries))
print('cold user:', len(cold_uids))
print('warm item:', len(iids))
print('cold item:', len(cold_iids))
print('avg. train item per user:', train_counter/len(train_ui))
print('avg. test item per user:', test_counter/len(test_ui))

random.shuffle(u_queries)
u_queries = u_queries[:2000]
rec_ui = []
with concurrent.futures.ProcessPoolExecutor(max_workers=8) as executor:
    for res in tqdm(executor.map(process_u_query, u_queries)):
        rec_ui.append(res)

print ('write the result to', "ml.lfm"+'.ui.rec')
with open("./exp/ml.lfm"+'.ui.rec', 'w') as f:
    f.write('%s\n' % ('\n'.join(rec_ui)))

In [26]:
from math import log

rec_file = "exp/ml.lfm.ui.rec"
at_k = 10

eval_num = 0.
eval_hit = 0.
eval_map = 0.
eval_recall = 0.
eval_ndcg = 0.

with open(rec_file, 'r') as f:
    for line in f:
        line = line.rstrip().split(' ')
        uid = line[0]
        len_ans = min(at_k, int(line[1]))
        if len_ans == 0: continue

        weight_res = list(map(float, line[2:]))[:at_k]
        binary_res = [min(v, 1) for v in weight_res]

        # scoring
        dcg, idcg = 0., 0.
        match, mAP = 0., 0.
        for pos, value in enumerate(weight_res, 1):
            if pos <= len_ans:
                idcg += 1./log(pos+1, 2)
            if value:
                dcg += value/log(pos+1, 2)
                match += 1
                mAP += match/pos
        eval_num += 1
        eval_hit += sum(binary_res)/len(binary_res)
        eval_map += mAP/len(binary_res)
        eval_ndcg += dcg/idcg
        eval_recall += sum(binary_res)/min(at_k, int(line[1]))

print ('===')
print ('   HIT@%d:\t%f' % (at_k, eval_hit/eval_num))
print ('   MAP@%d:\t%f' % (at_k, eval_map/eval_num))
print ('  NDCG@%d:\t%f' % (at_k, eval_ndcg/eval_num))
print ('RECALL@%d:\t%f' % (at_k, eval_recall/eval_num))
print ('===')

===
   HIT@10:	0.012450
   MAP@10:	0.004550
  NDCG@10:	0.013273
RECALL@10:	0.014471
===
